In [1]:
from dask import dataframe as dd
import matplotlib.pyplot as plt

In [2]:
ddf_users = dd.read_csv('..\\data\\raw\\2022_place_canvas_history_users.csv')

ddf_users.head(15)

,user_id
0,ovTZk4GyTS1mDQnTbV+vDOCu1f+u6w+CkIZ6445vD4XN8a...
1,6NSgFa1CvIPly1VniNhlbrmoN3vgDFbMSKqh+c4TTfrr3d...
2,O5Oityp3Z3owzTuwM9XnMggpLcqKEumsOMKGhRiDTTImWb...
3,tc273UiqS0wKa6VwiOs/iz/t4LyPYrhL2Q347awn11IQQE...
4,OOWsU/HLb4UUkQwclDeXFtsJTOXMlAdNHiRpFA1Qk+SxUr...
5,A0HdtcPvI7ipKivvXNVZDa3gkcjGXFjNxF5tca5QXazENC...
6,1U4LPuB22P6Yf7eRhKz6zU1dFMK5wXIzsNVPUNhP7eHIwG...
7,0AGoMGF50j0DJDc+704SwMylU90YDILDIgo8WOetgpiEWG...
8,tPcrtm7OtEmSThdRSWmB7jmTF9lUVZ1pltNv1oKqPY9bom...
9,7wfv1Tew3St1xYXQrTy7EF4LYwuuzxb0TXOxzxGBXriim8...


In [ ]:
active_users = ddf_users.user_id.value_counts().compute()

In [ ]:
most_active_users = active_users.head(20)
most_active_users

In [ ]:
plt.rcParams['font.size'] = '10'
plot = most_active_users.plot(figsize=(15,15), kind='barh')
plot.set_xscale('linear')
plot.set_xlabel('Number of placed pixels')
plot.set_ylabel('Unique identifier')
plot.set_title('Top 20 most active users')
plot.invert_yaxis()

In [4]:
least_active_users = ddf_users.user_id.value_counts().compute().reset_index(name='num')


In [14]:
num_least_active_users = least_active_users.num.value_counts().sort_values(ascending=False)
num_least_active_users.head(15)

1     2340498
2     1175052
3      768335
4      572021
5      460215
6      383663
7      330170
8      287842
9      256041
10     227403
11     204489
12     185885
13     169180
14     155614
15     143961
Name: num, dtype: int64

In [ ]:
#line graph for the above cell

In [ ]:
ddf_main = dd.read_csv('..\\data\\raw\\2022_place_canvas_history_optimized.csv')

ddf_main.head(15)

In [ ]:
most_used_colors = ddf_main.pixel_color.value_counts().compute()
most_used_colors

In [ ]:
plt.rcParams['font.size'] = '20'
plot = most_used_colors.plot(figsize=(15,15), kind='barh')
plot.set_xscale('symlog')
plot.set_xlabel('Number of occurances')
plot.set_ylabel('Color')
plot.set_title('Colors ranked by number of occurances')
plot.invert_yaxis()

In [ ]:
most_active_spots = ddf_main.coordinate.value_counts().compute()
top_active_spots = most_active_spots.head(20)
top_active_spots

In [ ]:
# To-do: Fix font size
plt.rcParams['font.size'] = '20'
plot = top_active_spots.plot(figsize=(15,15), kind='barh')
plot.set_xscale('linear')
plot.set_xlabel('Number of changes')
plot.set_ylabel('Coordinates')
plot.set_title('Top 20 most active spots')
plot.invert_yaxis()